## Do not change this notebooks 
We use it for preprocessing

In [3]:
import sys
import os

sys.path.insert(1, os.path.realpath(os.path.pardir))

import numpy as np
from pathlib import Path

import torch

from utils import data_utils
from utils import losses, hand_visualize

from utils.quats_and_angles import get_angles, get_quats
from einops import rearrange
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt

%matplotlib qt

In [3]:
def butter_highpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = signal.butter(order, normal_cutoff, btype="highpass")
    return b, a


def butter_highpass_filter(data, cutoff_freq, nyq_freq, order=4):
    b, a = butter_highpass(cutoff_freq, nyq_freq, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [4]:
config = dict(
    original_fps=200,  # TODO describtion
    delay_ms=0,  # Shift vr vs EMG parameter. ms dealy between emg and VR.
    start_crop_ms=0,  # bad values in the beginning of recordign in ms to delete.
    window_size=256,
)


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


config = AttrDict(config)


def get_subdirs(rootdir):
    paths = []
    for path in Path(rootdir).iterdir():
        if path.is_dir():
            paths.append(path)
    return paths

## Check one recording

In [ ]:
data = np.load(
    f"C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Middle1/1/1.0_data_test_662409.npz"
)

In [ ]:
one_c = data["data_myo"][:, 0]
plt.plot(one_c[~np.isnan(one_c)])

In [ ]:
print("EMG shape:", data["data_myo"].shape)
print("Time Stamps EMG shape:", data["myo_ts"].shape)
print("VR shape:", data["data_vr"].shape)
print("Time Stamps VR shape:", data["vr_ts"].shape)

In [ ]:
# plt.plot(data['myo_ts'])
plt.plot(data["vr_ts"])

In [ ]:
quats = data["data_vr"][:, :, 4:8]

In [ ]:
plt.plot(quats[:, 5, 0])

In [ ]:
emg = data["data_myo"]

In [ ]:
plt.plot(data["myo_ts"])

In [ ]:
vr = data["data_vr"][:, :, 4:8]

In [ ]:
one_channel = vr[:, 1, 3]
plt.plot(one_channel[~np.isnan(one_channel)])

# Get paths and preprocess raw datasets

In [5]:
participant = "Marusya"
ROOT_DIR = Path(
    f"C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/{participant}/GeneralTraining/"
)

DATASET_NAMES = ["Simple1"]

In [6]:
# get all paths to all folders inside above datasets
dataset_paths = [ROOT_DIR / Path(n) for n in DATASET_NAMES]
ALL_PATHS = []
for dp in dataset_paths:
    ALL_PATHS.extend(get_subdirs(dp))

print("ALL_PATHS: ", ALL_PATHS)
print("Number of paths: ", len(ALL_PATHS))


# Preprocess all datasets for angles extraction.

for path in ALL_PATHS:
    dataset = data_utils.create_dataset(
        data_folder=path,
        original_fps=config.original_fps,
        delay_ms=config.delay_ms,
        start_crop_ms=config.start_crop_ms,
        window_size=config.window_size,
        random_sampling=False,
        transform=None,
    )
    if len(dataset) == 0:
        print("WWWWW: Problem with dataset")
        break
    # go through each move and get angles and save.
    for idx, move in tqdm(enumerate(dataset.exps_data)):
        ts, myo, vr = move["myo_ts"], move["data_myo"], move["data_vr"]
        print("Shape:", vr.shape)
        angles = get_angles(vr)

        new_path = path.parents[0] / Path("preproc_angles") / Path(path.name)
        new_path.mkdir(parents=True, exist_ok=True)

        filename = f"{idx:04d}.npz"
        filepath = new_path / filename

        np.savez(filepath, data_myo=myo, data_vr=vr, data_angles=angles, myo_ts=ts)

ALL_PATHS:  [WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Marusya/GeneralTraining/Simple1/1'), WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Marusya/GeneralTraining/Simple1/2'), WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Marusya/GeneralTraining/Simple1/3'), WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Marusya/GeneralTraining/Simple1/4'), WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Marusya/GeneralTraining/Simple1/5'), WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Marusya/GeneralTraining/Simple1/preproc_angles')]
Number of paths:  6
Number of moves: 8 | Dataset: GeneralTraining



  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

Slice myo_timestamps and all data from 0 to 44985



 12%|██████████▌                                                                         | 1/8 [00:06<00:42,  6.14s/it]

Slice myo_timestamps and all data from 0 to 44979


...


0it [00:00, ?it/s]

Shape: (22493, 16, 4)



1it [00:25, 25.38s/it]

Shape: (22490, 16, 4)


1it [00:27, 27.48s/it]

KeyboardInterrupt



# Check the quality. How to use new data.

Here I visualize and check correctness of angle prediction. 

In [ ]:
rootdir = Path(
    f"C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/{participant}/GeneralTraining/{DATASET_NAMES[0]}/preproc_angles"
)


files = list(rootdir.glob("*/*"))


data = np.load(files[4])



emg, quats, angles = data["data_myo"], data["data_vr"], data["data_angles"]

In [ ]:
print("Preprocessed EMG:", emg.shape)
print("Preprocessed Quaternions:", quats.shape)
print("Preprocessed Angles:", angles.shape)

In [ ]:
plt.plot(angles[:, 6])

In [ ]:
plt.plot((np.sum(angles, axis=1) - 1) * 0.1)
plt.plot(np.abs(emg[:, 1]), alpha=0.6)
plt.xlabel("Samples", fontsize=12)
plt.ylabel("Normalized Data", fontsize=12)
plt.legend(["Sum of Angles", "Absolute EMG"], fontsize=12)

## Make gif of true kinematics

In [ ]:
NEW_FPS = 25
DRAW_EVERY = 250 // 25

visualize_and_save_anim_gifs(
    data=quats[:2000:DRAW_EVERY],
    path=Path("C:/Users/vlvdi/Desktop/EMG/original_quats.gif"),
    fps=NEW_FPS,
)

## Quats to Angle transform check

In [ ]:
from utils.hand_visualize import (
    Hand,
    save_animation_mp4,
    visualize_and_save_anim,
    merge_two_videos,
    visualize_and_save_anim_gifs,
)


def get_angle_degree(y_hat, y_batch):
    """
    [batch, n_bones, 4, time]
    """
    (
        time,
        n_bones,
        n_quat,
    ) = y_hat.shape

    y_hat, y_batch = y_hat.reshape(-1, 4), y_batch.reshape(-1, 4)

    mult = torch.sum(y_hat * y_batch, dim=-1) ** 2

    angle_degree = torch.mean(
        torch.arccos(torch.clip((2 * mult - 1), -1, 1)) / torch.pi * 180
    )


    return angle_degree.item()

In [ ]:
files = list(rootdir.glob("*/*"))
data = np.load(files[0])

emg, quats, angles = data["data_myo"], data["data_vr"], data["data_angles"]
quats_hat = get_quats(angles)
angle_diff = get_angle_degree(torch.from_numpy(quats_hat), torch.from_numpy(quats))

print("Shapes", emg.shape, quats.shape, angles.shape, quats_hat.shape)
print("Angle degree", angle_diff)

In [ ]:
NEW_FPS = 25
DRAW_EVERY = 250 // 25

visualize_and_save_anim_gifs(
    data=quats[:2000:DRAW_EVERY],
    path=Path("C:/Users/vlvdi/Desktop/EMG/original_quats.gif"),
    fps=NEW_FPS,
)

In [ ]:
visualize_and_save_anim_gifs(
    data=quats_hat[:2000:DRAW_EVERY],
    path=Path("C:/Users/vlvdi/Desktop/EMG/11_15_2023/from_angles_quats.gif"),
    fps=NEW_FPS,
)